In [32]:

from kinetics_modelling.config import load_config, PROCESSED_DATA_DIR
import polars as pl
import altair as alt
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [ ]:
condig = load_config()

TypeError: load_config() missing 1 required positional argument: 'config_name'

In [13]:
load_config('ob006-run0.ini')['Paths']['bam_filepath']

'~/mutationalscanning/DerivedData/bam/HiFi/human/ob006/kinetics/ob006_kinetics_diploid.bam'

In [14]:
from pathlib import Path
Path(config.load_config('ob006.ini')['Paths']['bam_filepath']).name.split('_')[0]

FileNotFoundError: Config file not found at: /Users/charlton/projects/kinetics_modelling/configs/ob006.ini

In [29]:
pl.Config(fmt_str_lengths=50)
df = pl.read_parquet(PROCESSED_DATA_DIR / 'sample_bam_null')#.filter(pl.col('center_qual')==[93,93,93])
df = df.with_columns(
    (pl.col('center_qual').list[0].alias('center_qual_0')),
    (pl.col('center_qual').list[1].alias('center_qual_1')),
    (pl.col('center_qual').list[2].alias('center_qual_2'))
)
df.head()

contig,read_name,read_is_reverse,fn,rn,ref_center,read_center,center_seq,center_qual,center_ipd_fwd_0,center_ipd_fwd_1,center_ipd_fwd_2,center_ipd_rev_0,center_ipd_rev_1,center_ipd_rev_2,center_pw_fwd_0,center_pw_fwd_1,center_pw_fwd_2,center_pw_rev_0,center_pw_rev_1,center_pw_rev_2,window_sm,window_sx,window_seq,window_qual,window_ipd_fwd,window_ipd_rev,window_pw_fwd,window_pw_rev,center_qual_0,center_qual_1,center_qual_2
str,str,bool,i64,i64,i64,i64,str,list[u8],u16,u16,u16,u16,u16,u16,u8,u8,u8,u8,u8,u8,list[u16],list[u16],list[str],list[u8],list[u16],list[u16],list[u8],list[u8],u8,u8,u8
"""h1tg000001l""","""m84108_240530_160720_s3/92602827/ccs""",false,4,4,16539013,160,"""GAA""","[93, 80, 93]",16,13,19,18,10,16,33,16,12,43,18,13,"[10, 9, … 10]","[0, 1, … 0]","[""G"", ""A"", … ""T""]","[93, 87, … 93]","[26, 12, … 38]","[23, 59, … 14]","[13, 11, … 28]","[40, 17, … 12]",93,80,93
"""h1tg000001l""","""m84108_240530_160720_s3/92602827/ccs""",false,4,4,16539077,224,"""AAA""","[51, 93, 93]",2,16,5,9,5,5,39,8,12,15,32,25,"[10, 9, … 10]","[0, 0, … 0]","[""A"", ""C"", … ""C""]","[93, 88, … 53]","[12, 10, … 21]","[32, 39, … 10]","[24, 19, … 41]","[22, 25, … 33]",51,93,93
"""h1tg000001l""","""m84108_240530_160720_s3/92602827/ccs""",false,4,4,16539141,288,"""AAT""","[63, 93, 82]",10,9,12,8,18,14,12,18,11,20,16,21,"[10, 10, … 9]","[0, 0, … 0]","[""A"", ""C"", … ""T""]","[93, 67, … 93]","[10, 16, … 13]","[8, 8, … 13]","[9, 23, … 19]","[20, 21, … 20]",63,93,82
"""h1tg000001l""","""m84108_240530_160720_s3/92602827/ccs""",false,4,4,16539205,352,"""TCT""","[93, 83, 67]",22,7,9,18,5,8,18,34,21,40,28,11,"[10, 10, … 10]","[0, 0, … 0]","[""T"", ""T"", … ""C""]","[93, 93, … 84]","[7, 12, … 10]","[11, 11, … 10]","[29, 38, … 30]","[28, 26, … 27]",93,83,67
"""h1tg000001l""","""m84108_240530_160720_s3/92602827/ccs""",false,4,4,16539269,416,"""TAT""","[93, 83, 91]",42,68,100,6,19,9,18,12,9,9,7,23,"[9, 8, … 10]","[0, 1, … 0]","[""T"", ""T"", … ""A""]","[77, 93, … 93]","[13, 9, … 5]","[10, 52, … 7]","[12, 18, … 25]","[12, 32, … 11]",93,83,91


In [47]:
df_passes_group = df.group_by('fn').agg(
    pl.col('center_qual_1').var().alias('center_qual_1_var'),
    pl.col('center_qual_1').mean().alias('center_qual_1.mean')
    )
df_passes_group.head()

fn,center_qual_1_var,center_qual_1.mean
i64,f64,f64
3,313.728575,79.890073
6,108.139185,89.290355
7,59.839914,90.896947
1,334.370115,47.327402
4,236.351053,84.651406


In [49]:

alt.Chart(df).mark_circle().encode(
    alt.X('fn'),
    alt.Y('center_qual_1:Q')
)

alt.Chart(...)

In [ ]:
chart_fn_variance = alt.Chart(df_passes_group).mark_circle().encode(
    alt.X('fn:Q'),
    alt.Y('center_qual_1_var:Q')
) 
chart_fn_mean = 

alt.Chart(...)